In [ ]:
import numpy as np
import pandas as pd
import os
from scipy import stats
from scipy.stats import ttest_1samp
import matplotlib.pyplot as plt

In [ ]:
Data_path = r"Y:\Projects\2023_Scholte_FMG1441\Data"
epochs_timewindows = np.load(os.path.join(Data_path, "Epochs_Time_Window.npy"))
# Set the plot size
plt.rcParams['figure.figsize'] = [18, 12]
max_sub = 15    # Maximum number of participants
min_rep = 5     # Define minimum repetition
ce = []
sc = []
ce_pvalue = []
sc_pvalue = []
ce_confidence_interval = []
sc_confidence_interval = []

In [ ]:
for i in range(5, max_sub+1):
    sub_name = "sub_" + str(i)
    sub_data_path = os.path.join(Data_path, sub_name, "Regression", sub_name + "_βcoef(Oz).npy")
    sub_data = np.load(sub_data_path)
    ce.append(list(sub_data[:, 0:min_rep].mean(axis = 1)))
    # ce += list(sub_data[:, 0:min_rep].mean(axis = 1))
    sc.append(list(sub_data[:, min_rep:].mean(axis = 1)))
    print(i)
    # sc += list(sub_data[:, min_rep:].mean(axis = 1))
ce = np.array(ce)
ce.shape
sc = np.array(sc)
sc.shape

In [ ]:
for i in range(0, ce.shape[1]):
    ce_results = stats.ttest_1samp(ce[:, i], popmean = 0)
    ce_ci = ce_results.confidence_interval(confidence_level=0.95)
    ce_pvalue.append(ce_results.pvalue)
    ce_confidence_interval.append(list(ce_ci))
    sc_results = stats.ttest_1samp(sc[:, i], popmean = 0)
    sc_ci = sc_results.confidence_interval(confidence_level=0.95)
    sc_pvalue.append(sc_results.pvalue)
    sc_confidence_interval.append(list(sc_ci))
ce_pvalue = np.array(ce_pvalue)
sc_pvalue = np.array(sc_pvalue)
ce_confidence_interval = np.array(ce_confidence_interval)
sc_confidence_interval = np.array(sc_confidence_interval)
min_coef = np.min((np.min(ce_confidence_interval), np.min(sc_confidence_interval)))

In [ ]:
plt.figure(2).clear()
plt.figure(2)
ce_plt, = plt.plot(epochs_timewindows, ce.mean(axis = 0), label = "CE")
plt.fill_between(epochs_timewindows, ce_confidence_interval[:, 0], ce_confidence_interval[:, 1], alpha = .1)
sc_plt, = plt.plot(epochs_timewindows, sc.mean(axis = 0), label = "SC")
plt.fill_between(epochs_timewindows, sc_confidence_interval[:, 0], sc_confidence_interval[:, 1], alpha = .1)
ce_p_index = np.where(ce_pvalue < 0.05)[0]
sc_p_index = np.where(sc_pvalue < 0.05)[0]
plt.scatter(epochs_timewindows[ce_p_index], [min_coef] * ce_p_index.shape[0], marker = "|", c = ce_plt.get_c())
plt.scatter(epochs_timewindows[sc_p_index], [min_coef*0.98] * sc_p_index.shape[0], marker = "|", c = sc_plt.get_c())
plt.axhline(y=0, color='r', linestyle='-')
plt.xlabel('Time')
plt.ylabel('β-coefficients')
plt.title("β-coefficients T test")
plt.legend(fontsize="20")
plt.savefig(os.path.join(Data_path, "Coefficients_T-test.png"))